<span style="color:#888888">Copyright (c) 2014-2025 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Tutorial 1:**</span> <span style="color:#555555">How to Create Trajectory Points from a Delimited File</span>

## Purpose

This notebook demonstrates how to create Tracktable Trajectory Point objects from a delimited text file (comma-separated, tab-separated, et cetera).  A data file must contain the following columns in order to be compatible with Tracktable:

* **<span style="color:#00add0">an identifier</span>** that is unique to each object
* **<span style="color:#00add0">a timestamp</span>**
* **<span style="color:#00add0">longitude</span>**
* **<span style="color:#00add0">latitude</span>**

Both ordering and headers for these columns can vary, but they must exist in the file.  Each row of the data file should represent the information for a single trajectory point.  

**<span style="color:#81062e">IMPORTANT:</span>** delimited files must be **sorted by timestamp in increasing order** to be compatible with Tracktable.

*Note:* This notebook does not cover how to create a Trajectory object (as opposed to a list of Trajectory point objects).  Please see [Tutorial 2](Tutorial_02_Points_From_File.ipynb) for an example of how to create Trajectory objects from a csv file containing trajectory point information.

## Step 1: Identify your CSV/TSV File

We will use the provided example data $^1$ for this tutorial.  If you are using another filename, `data_filename` should be set to the string containing the path to your csv file.

In [ ]:
from tracktable_data.data import retrieve

data_filename = retrieve(filename='NYHarbor_2020_06_30_first_hour.csv')

## Step 2: Create a TrajectoryPointReader object.

We will create a Terrestrial point reader, which will expect **(longitude, latitude)** coordinates.  Alternatively, if our data points were in a Cartesian coordinate system, we would import the `TrajectoryPointReader` object from `tracktable.domain.cartesian2d` or `tracktable.domain.cartesian3d`.

In [ ]:
from tracktable.domain.terrestrial import TrajectoryPointReader

reader = TrajectoryPointReader()

## Step 3: Give the TrajectoryPointReader an input source.

This must be a Python file-like object.  Files opened with `open()` are the most common use case.

In [ ]:
reader.input = open(data_filename, 'r')

### <span style="color:#0f0f0f">*Additional Settings*</span>

Identify the comment character for the data file.  Any lines with this as the first non-whitespace character will be ignored.  This is optional and defaulted to `#`.

In [ ]:
reader.comment_character = '#'

Identify the file's delimiter.  For comma-separated (CSV) files, the delimiter should be set to `,`.  For tab-separated files, this should be `\t`.  This is optional, and the default value is `,`.  If your field delimiter is some other character, substitute it here.

In [ ]:
reader.field_delimiter = ','

Identify the string associated with a null value in a cell.  This is optional and defaults to an empty string.

In [ ]:
reader.null_value = 'NaN'

### <span style="color:#0f0f0f">*Required Columns*</span>

We must tell the reader where to find the **<span style="color:#00add0">unique object ID</span>**, **<span style="color:#00add0">timestamp</span>**, **<span style="color:#00add0">longitude</span>** and **<span style="color:#00add0">latitude</span>** columns.  Column numbering starts at zero.

If no column numbers are given, the reader will assume they are in the order listed above: object ID in column 0, timestamp in column 1, longitude in column 2, latitude in column 3.  

**Note**: Tracktable stores geodetic (terrestrial) coordinates with the longitude first and latitude second.


In [ ]:
reader.object_id_column = 3
reader.timestamp_column = 0
reader.coordinates[0] = 1     # longitude
reader.coordinates[1] = 2     # latitude

### <span style="color:#0f0f0f">*Optional Columns*</span>

Your data file may contain additional information (e.g. speed, heading, altitude, etc.) that you wish to store with your trajectory points.  These can be stored as either floats, strings or datetime objects.  An example of each is shown below, respectively.

In [ ]:
reader.set_real_field_column('heading', 6)
reader.set_string_field_column('vessel-name', 7)
reader.set_time_field_column('eta', 17)

## Step 4: Convert the Reader to a List of Trajectory Points

`TrajectoryPointReader` functions as an iterable of points.  Once an instance has been configured with an input source and a list of fields.  

The sometimes-inconvenient thing about iterables is that they can only be traversed once.  Here we store all the points in a list so that we can access them at will.

In [ ]:
trajectory_points = list(reader)

How many trajectory points do we have?

In [ ]:
len(trajectory_points)

## Step 5: Accessing Trajectory Point Info

The information from the required columns of the csv can be accessed for a single `TrajectoryPoint` object as

* **<span style="color:#00add0">unique object identifier:</span>** `trajectory_point.object_id`
* **<span style="color:#00add0">timestamp:</span>** `trajectory_point.timestamp`
* **<span style="color:#00add0">longitude:</span>** `trajectory_point[0]`
* **<span style="color:#00add0">latitude:</span>** `trajectory_point[1]`

The optional column information is available through the member variable `properties` as follows: `trajectory_point.properties['what-you-named-it']`.

This is demonstrated below for our first ten trajectory points.

In [ ]:
for traj_point in trajectory_points[:10]:
    object_id    = traj_point.object_id
    timestamp    = traj_point.timestamp
    longitude    = traj_point[0]
    latitude     = traj_point[1]
    heading      = traj_point.properties["heading"]
    vessel_name  = traj_point.properties["vessel-name"]
    eta          = traj_point.properties["eta"]
    
    print(f'Unique ID: {object_id}')
    print(f'Timestamp: {timestamp}')
    print(f'Longitude: {longitude}')
    print(f'Latitude: {latitude}')
    print(f'Heading: {heading}')
    print(f'Vessel Name: {vessel_name}')
    print(f'ETA: {eta}\n')

<span style="color:gray">$^1$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. *AIS Data for 2020.* Retrieved February 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  Trimmed down to the first hour of June 30, 2020, restricted to in NY Harbor.</span>